**Ejercicios Planteados**

La base de datos House_price.csv contiene información de casas. La idea es obtener un modelo que permita predecir el valor de venta de una casa. Las variables son las siguientes:

• LotFrontage: Pies lineales de calle conectados a la propiedad

• LotArea: Tamaño del lote en pies cuadrados

• MasVnrArea: Área de revestimiento de mampostería en pies cuadrados

• BsmtFinSF1: Pies cuadrados del sótano terminados tipo 1

• BsmtFinSF2: Pies cuadrados con del sótano acabados tipo 2

• BsmtUnfSF: Pies cuadrados sin terminar de área del sótano

• TotalBsmtSF: Total de pies cuadrados de área del sótano

• 1stFlrSF: Pies cuadrados del primer piso

• 2ndFlrSF: Pies cuadrados del segundo piso

• LowQualFinSF: Pies cuadrados con acabado de baja calidad (todos los pisos)

• GrLivArea: Pies cuadrados de área habitable sobre el nivel (suelo)

• GarageArea: Tamaño del garaje en pies cuadrados

• WoodDeckSF: Área de la plataforma de madera en pies cuadrados

• OpenPorchSF: Área del porche abierto en pies cuadrados

• EnclosedPorch: Área del porche cerrado en pies cuadrados

• 3SsnPorch: área de porche de tres estaciones en pies cuadrados

• ScreenPorch: Área del porche de la pantalla en pies cuadrados

• PoolArea: Área de la piscina en pies cuadrados

• MiscVal: Valor de la función miscelánea

• SalePrice: el precio de venta de la propiedad en dólares. Esta es la variable objetivo que se quiere modelar.


In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from datetime import datetime

### a) Cargue la base de datos, ¿hay alguna columna que no sea útil para el análisis? Revise si existen casos faltantes en la base de datos, según la descripción de las variables, ¿qué pudiera significar un NA? Tome decisiones de qué hacer si existen casos faltantes. Si desea imputar los valores nulos de alguna variable por un valor puede utilizar data["columna"].fillna('valor', inplace = True).

In [9]:
#ruta del data
house_price = "house_price.csv"

# Cargar la base de datos
data = pd.read_csv(house_price, delimiter=';')

# Imprimir las primeras filas para ver qué columnas hay
print(data.head())

# Verificar si existen casos faltantes
print(data.isnull().sum())

# Eliminar la columna "Id" (no útil para el análisis)
data.drop(columns=["Id"], inplace=True)

# Tratar valores faltantes (NA)
# Imputar los valores nulos en todas las columnas con un valor específico (por ejemplo, "valor")
data = data.apply(lambda x: x.fillna(x.mean()) if x.dtype.kind in 'biufc' else x.fillna('valor'))

   Id  LotFrontage  LotArea  MasVnrArea  BsmtFinSF1  BsmtFinSF2  BsmtUnfSF  \
0   1         65.0     8450       196.0         706           0        150   
1   2         80.0     9600         0.0         978           0        284   
2   3         68.0    11250       162.0         486           0        434   
3   4         60.0     9550         0.0         216           0        540   
4   5         84.0    14260       350.0         655           0        490   

   TotalBsmtSF  1stFlrSF  2ndFlrSF  ...  GrLivArea  GarageArea  WoodDeckSF  \
0          856       856       854  ...       1710         548           0   
1         1262      1262         0  ...       1262         460         298   
2          920       920       866  ...       1786         608           0   
3          756       961       756  ...       1717         642           0   
4         1145      1145      1053  ...       2198         836         192   

   OpenPorchSF  EnclosedPorch  3SsnPorch  ScreenPorch  PoolAre

### b) La variable a modelar corresponde al precio de venta de la casa en dólares. En base a los modelos que hemos aprendido en clases, ¿qué modelo(s) podría(n) ser de utilidad para predecir el precio de venta de casas mediante las variables predictoras?

Como buscamos anticipar el precio de venta de viviendas, resulta idóneo emplear modelos de regresión. Entre las opciones útiles se incluyen: regresión lineal, regresión Ridge, regresión Lasso y regresión de árboles (como Random Forest).

### c) Vamos a implementar un modelo de regresión lineal para modelar el precio de venta de casas. Considerando que en la base de datos tenemos varias variables que dependen de otras, ¿cuál pudiera ser el problema que surgiría en este contexto? ¿por qué esto es un problema? Discuta.

En este caso podría surgir en este contexto un problema de dependencia entre variables.

La multicolinealidad entre las variables predictoras plantea un desafío en el análisis de regresión, surgiendo cuando dos o más variables independientes muestran una correlación significativa entre sí. Algunos aspectos fundamentales de este problema incluyen:

Interpretación de Coeficientes: En un modelo de regresión lineal, los coeficientes indican el cambio en la variable dependiente (como el precio de venta) por cada unidad de cambio en una variable independiente. Sin embargo, cuando dos variables predictoras están altamente correlacionadas, resulta complicado discernir cuál de ellas está ejerciendo realmente influencia sobre la variable objetivo.
Inestabilidad de Coeficientes: La presencia de multicolinealidad puede generar inestabilidad en los coeficientes del modelo, volviéndolos susceptibles a pequeñas variaciones en los datos. Este fenómeno dificulta tanto la interpretación como la fiabilidad del modelo.

### d) Realice un análisis de la correlación entre las variables. Comente, ¿existen variables muy correlacionadas entre sí? ¿Por qué cree que ocurre?

In [10]:
# Calcular la matriz de correlación
correlation_matrix = data.corr()

# Filtrar las correlaciones fuertes (mayores que 0.7 o menores que -0.7)
strong_correlations = correlation_matrix[(correlation_matrix > 0.7) | (correlation_matrix < -0.7)]

# Imprimir las correlaciones fuertes
print(strong_correlations)

               LotFrontage  LotArea  MasVnrArea  BsmtFinSF1  BsmtFinSF2  \
LotFrontage            1.0      NaN         NaN         NaN         NaN   
LotArea                NaN      1.0         NaN         NaN         NaN   
MasVnrArea             NaN      NaN         1.0         NaN         NaN   
BsmtFinSF1             NaN      NaN         NaN         1.0         NaN   
BsmtFinSF2             NaN      NaN         NaN         NaN         1.0   
BsmtUnfSF              NaN      NaN         NaN         NaN         NaN   
TotalBsmtSF            NaN      NaN         NaN         NaN         NaN   
1stFlrSF               NaN      NaN         NaN         NaN         NaN   
2ndFlrSF               NaN      NaN         NaN         NaN         NaN   
LowQualFinSF           NaN      NaN         NaN         NaN         NaN   
GrLivArea              NaN      NaN         NaN         NaN         NaN   
GarageArea             NaN      NaN         NaN         NaN         NaN   
WoodDeckSF             Na

TotalBsmtSF y 1stFlrSF presentan una correlación positiva notable de 0.82, lo cual no sorprende dado que el área total del sótano típicamente abarca el espacio del primer piso. En cuanto a GrLivArea y 2ndFlrSF, la correlación positiva de 0.69 refleja la tendencia de que el área habitable sobre el suelo incluya comúnmente el segundo piso. Por último, la sólida correlación positiva de 0.71 entre GrLivArea y SalePrice indica que, en general, las propiedades más espaciosas tienden a tener precios de venta más altos.

### e) Suponga que este es un problema de gran volumen, donde tenemos muchas columnas y registros. Cuando existen variables muy correlacionadas el costo computacional para obtener el modelo puede ser muy elevado. Obtenga el tiempo de procesamiento para obtener una regresión lineal. Para esto, podemos obtener la regresión lineal utilizando la función LinearRegression de sklearn.linear_model y calcular el tiempo de procesamiento de la siguiente forma:

from datetime import datetime
start=datetime.now()
[aquí va el código del modelo]
print(datetime.now()-start) #esto entrega el tiempo que demoró
en correr el modelo.


In [11]:
# Separar variables predictoras (X) y variable objetivo (y)
X = data.drop(columns=["SalePrice"])
y = data["SalePrice"]

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Estandarizar las variables
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Obtener el tiempo de procesamiento para la regresión lineal
start = datetime.now()
regression_model = LinearRegression()
regression_model.fit(X_train_scaled, y_train)
print("Tiempo de procesamiento para la regresión lineal:", datetime.now() - start)

Tiempo de procesamiento para la regresión lineal: 0:00:00.004193


### f) Obtenga las componentes principales utilizando las variables estandarizadas, si desea explicar un 80% de la variabilidad, ¿cuántas componentes debería elegir? Argumente.

In [12]:
# Obtener las componentes principales
pca = PCA(n_components=0.8)  # Explicar al menos el 80% de la variabilidad
X_train_pca = pca.fit_transform(X_train_scaled)

### g) Corra el modelo de regresión lineal con las componentes principales como variables predictoras. Vuelva a calcular el tiempo de procesamiento. ¿En cuál caso suele demorarse menos? ¿Qué ganamos al realizar componentes principales? Concluya.

In [13]:
# Correr el modelo de regresión lineal con las componentes principales
start_pca = datetime.now()
regression_model_pca = LinearRegression()
regression_model_pca.fit(X_train_pca, y_train)
print("Tiempo de procesamiento con componentes principales:", datetime.now() - start_pca)

Tiempo de procesamiento con componentes principales: 0:00:00.003100


Reducción de dimensionalidad: Utilizando PCA, podemos disminuir el número de variables originales (dimensiones) en nuestros datos. Al elegir un conjunto reducido de componentes principales, simplificamos el modelo y aumentamos la eficiencia computacional.

Eliminación de multicolinealidad: Dado que las componentes principales son ortogonales entre sí, ayudan a mitigar la multicolinealidad entre las variables originales. Esto facilita la interpretación de los coeficientes en los modelos de regresión.

Conservación de la variabilidad: A pesar de la reducción de dimensionalidad, el PCA se esfuerza por preservar la mayor parte posible de la variabilidad original en los datos. Al seleccionar un número adecuado de componentes principales, podemos explicar una parte significativa de la variabilidad total.

En resumen, mediante el análisis de componentes principales, obtenemos una representación más eficaz de nuestros datos, reduciendo la complejidad mientras mantenemos la información esencial. No obstante, es importante equilibrar este proceso con la cantidad de variabilidad que estamos dispuestos a sacrificar.